In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()

import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

(cont:sync:readmostly)=
# Read-Dominated Workloads

There are many scenarios in operating systems and other concurrent software where certain data structures are frequently read, but rarely updated. For example, the OS caches network routing information, but route updates are infrequent compared to route lookups in the cache. Since read-only accesses cannot conflict with each other, we would like to allow "readers" to execute concurrently with each other (sharing access to the data structure), while "writers" must not be allowed to conflict with either readers or other writers. We will look at two ways to solve this problem: classical reader-writer locks (rw_locks), and read-copy-update (RCU). 

(cont:sync:readmostly:rw_locks)=
## Reader-Writer Locks

Courtois et al. first described the reader-writer problem and two solutions using semaphores for mutual exclusion in 1971 {cite}`10.1145/362759.362813`. In the first version of the problem, readers have priority over writers: a reader thread should never be prevented from accessing the resource unless a writer has already been granted exclusive use of the resource. We show the solution for this reader-priority version in {numref}`listing:sync:readmostly:readerpri` using POSIX semaphores. Note that since the semaphores are used only for mutual exclusion, we could replace them with `pthread_mutex_t` or any of our spinlock implementations without making any other changes. We represent access to some shared database with the `read_db()` and `write_db()` functions. 

```{literalinclude} /src/sync/readerpri.c
:linenos:
:emphasize-lines: 4-6, 10, 12, 18-24, 28-34
:start-after: Begin Courtois
:end-before: End Courtois
:name: listing:sync:readmostly:readerpri
:caption: Reader/writer solution using semaphores; readers have priority over writers.
```

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/readerpri.c", 
    lang="", 
    number=True,
    title="<b>readerpri.c - Full source code for readers/writers solution with semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>readerpri.c - Full source code for readers/writers solution with semaphores</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
 1: #include <pthread.h>
 2: #include <stdlib.h>
 3: #include <sys/types.h>
 4: #include <unistd.h>
 5: #include <string.h>
 6: #include <stdio.h>
 7: #include <time.h>
 8: #include <bsd/sys/time.h>
 9: #include <stdbool.h>
10: #include <stdatomic.h>
11: #include <semaphore.h>
12: 
13: atomic_bool started = false;
14: 
15: /* Begin Courtois et al. reader/writer semaphore solution with reader priority here */
16: static void read_db();
17: static void write_db();
18: 
19: sem_t mutex;         /* Initially 1 */
20: sem_t counter_mutex; /* Initially 1 */
21: int reader_count = 0;
22: 
23: static void writer()
24: {
25:     sem_wait(&mutex);
26:     write_db();
27:     sem_post(&mutex);
28: }
29: 
30: static void reader()
31: {
32:     
33:     sem_wait(&counter_mutex);
34:     reader_count++;
35:     if (reader_count == 1) {
36:         /* first reader synchronizes with writers to acquire the resource */
37:         sem_wait(&mutex);
38:     }
39:     sem_post(&counter_mutex);
40:     
41:     read_db();
42:     
43:     sem_wait(&counter_mutex);
44:     reader_count--;
45:     if (reader_count == 0) {
46:         /* last reader releases the resource */
47:         sem_post(&mutex);
48:     }
49:     sem_post(&counter_mutex);
50: }
51: /* End Courtois et al. reader/writer semaphore solution with reader priority. */
52: 
53: static void *writer_thread(void *arg)
54: {
55:     long niters = (long)arg;
56:     long i;
57:     
58:     while(!started) ; /* spin until all threads are ready */
59:         
60:     for (i = 0; i < niters; i++) {
61:         writer();
62:     }
63:     return (void *)0;
64: }
65: 
66: static void *reader_thread(void *arg)
67: {
68:     long niters = (long)arg;
69:     long i;
70:     
71:     while(!started) ; /* spin until all threads are ready */
72: 
73:     for (i = 0; i < niters; i++) {
74:         reader();
75:     }
76:     return (void *)0;
77: }
78: 
79: int main(int argc, char **argv)
80: {
81:     pthread_t *tids;
82:     long niters;
83:     int nreaders, nwriters, nthreads, i;
84:     
85:     if (argc != 4) {
86:         fprintf(stderr,"Usage: %s <num_readers> <num_writers> <num_iters>\n",argv[0]);
87:         return 1;
88:     }
89: 
90:     nreaders = atoi(argv[1]);
91:     nwriters = atoi(argv[2]);
92:     niters = atol(argv[3]);
93:     
94:     printf("Main thread: Beginning test with %d readers and %d writers, each doing %ld iterations\n", 
95:            nreaders, nwriters, niters);
96:     sem_init(&counter_mutex, 0, 1);
97:     sem_init(&mutex, 0, 1);
98:     
99:     nthreads = nreaders + nwriters;
100:     tids = (pthread_t *)malloc(nthreads * sizeof(pthread_t));
101:     
102:     /* We create the requested number of reader and writer threads, each doing the same number of iterations. 
103:      * When all threads have completed, we report the throughput. 
104:      */
105:     for (i = 0; i < nreaders; i++) {
106:         (void)pthread_create(&tids[i], NULL, reader_thread, (void *)niters );
107:     }
108:     
109:     for(i = 0; i < nwriters; i++) {
110:         (void)pthread_create(&tids[i+nreaders], NULL, writer_thread, (void *)niters );
111:     }
112:     
113:     struct timespec strt;
114:     struct timespec end;
115:     clock_gettime(CLOCK_MONOTONIC_RAW, &strt);
116:     started = true;
117:     /* Wait for child threads to finish */
118:     for (i = 0; i < nthreads; i++) {
119:         pthread_join(tids[i], NULL);
120:     }
121:     clock_gettime(CLOCK_MONOTONIC_RAW, &end);
122:     
123:     /* report performance statistics */    
124:     struct timespec diff;
125:     timespecsub(&end, &strt, &diff);
126:     float elapsed_sec = diff.tv_sec + diff.tv_nsec / 1000000000.0;
127:     printf("Main thread: Completed %ld iterations in %f seconds (%f iters per second)\n",
128:           niters*nthreads, elapsed_sec, niters*nthreads*1.0/elapsed_sec);
129: 
130:     return 0;
131: }
132: 
133: static void read_db() 
134: {
135:     return;
136: }
137: 
138: static void write_db()
139: {
140:     return;
141: }
```


</div>
</font>

The writer code (lines 11-13) is very simple. We introduce a `mutex` semaphore (line 4) to protect access to the shared database. The `wait` on `mutex` at line 10 ensures that only one writer can be accessing the database at any time; when the writer is done, it `post`s a notice to `mutex` that the database is now available again (line 12). The reader code needs to do a little more work, however, because we want to allow multiple readers. First, we want to keep track of the number of readers currently accessing the database, so we introduce a `reader_count` variable (line 6), and since this variable will be updated by every reader thread, we also add a `counter_mutex` semaphore (line 5) to protect sections of code that depend on the `reader_count`. Each reader thread begins by `wait`ing for the `counter_mutex`, then increments `reader_count` to add itself to the count of current readers (lines 18-19). Only the first reader thread needs to synchronize with writer threads (lines 20-23)---once a reader has successfully `wait`ed for the `mutex`, writers will be locked out until all the readers are done. Notice also that the first reader does not `post` to the `counter_mutex` until *after* it has obtained permission to access the database (line 24). This ensures that other reader threads must line up `wait`ing for the `counter_mutex` as long as a writer is accessing the database; once the first reader obtains access to the database, the other readers only need to increment the `reader_count`. When a reader finishes using the database, it must again `wait` for the `counter_mutex` to subtract itself from the count of current readers (lines 28-29). When the `reader_count` falls to zero, it means that the last reader thread is done using the database, and this last thread must `post` a signal to the `mutex` to indicate that the database is available again (lines 30-33). If any writers are waiting, one will now be allowed to proceed; otherwise the database remains available until either a reader or writer threads requests access.

The reader code illustrates the *lightswitch pattern* in concurrent programming: the analogy is that the first person into a dark room turns on the light (acquires the resource), and the last one to leave turns it off (releases the resource). 

We can encapsulate the *entry* sections of code (line 10 for writers; lines 18-24 for readers) and the *exit* sections of code (line 12 for writers; lines 28-34 for readers) into acquire and release functions for reader/writer locks. This code is shown in {numref}`listing:sync:readmostly:rwlock`:

```{literalinclude} /src/sync/rwlock.h
:linenos:
:name: listing:sync:readmostly:rwlock
:caption: C source code for reader/writer locks using semaphores.
```


In [4]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/rwlock.h", 
    lang="", 
    number=True,
    title="<b>rwlock.h - Source code for reader/writer locks using semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>rwlock.h - Source code for reader/writer locks using semaphores</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
 1: #include <semaphore.h>
 2: 
 3: typedef struct rwlock_s {
 4:     int reader_count;
 5:     sem_t counter_mutex;
 6:     sem_t mutex;
 7: } rwlock_t; 
 8: 
 9: static inline void rwlock_init(rwlock_t *l) {
10:     l->reader_count = 0;
11:     sem_init(&l->counter_mutex, 0, 1);
12:     sem_init(&l->mutex, 0, 1);
13: }
14: 
15: static inline void rwlock_acquire_write(rwlock_t *l)
16: {
17:     sem_wait(&l->mutex);
18: }
19: 
20: static inline void rwlock_release_write(rwlock_t *l)
21: {
22:     sem_post(&l->mutex);
23: }
24: 
25: static inline void rwlock_acquire_read(rwlock_t *l)
26: {
27:     sem_wait(&l->counter_mutex);
28:     l->reader_count++;
29:     if (l->reader_count == 1) {
30:         /* first reader synchronizes with writers to acquire the resource */
31:         sem_wait(&l->mutex);
32:     }
33:     sem_post(&l->counter_mutex);   
34: }
35: 
36: static inline void rwlock_release_read(rwlock_t *l)
37: {
38:     sem_wait(&l->counter_mutex);
39:     l->reader_count--;
40:     if (l->reader_count == 0) {
41:         /* last reader releases the resource */
42:         sem_post(&l->mutex);
43:     }
44:     sem_post(&l->counter_mutex);
45: }

```


</div>
</font>

We could combine the release functions and just have a single `rwlock_release()` function invoked by either reader or writer threads. When a writer releases the lock, the `reader_count` should already be `0`; in that case we skip decrementing `reader_count` and just post to the `mutex`. This is the approach taken by the pthreads library implementation of reader/writer locks (`man -k pthread_rwlock`). 

In this implementation of rwlocks, intensive read activity could starve writer threads. An alternative is to give writers priority: as soon as a writer thread requests access to the shared data, no new reader threads are allowed access until the writer is done using the data. The writer must still wait for existing reader threads to finish and release the data, but writers will not starve. If write activity is intense enough to worry about starving readers, then rwlocks are probably not a good choice anyway. 

## Performance of rwlocks

To be added. 

(cont:sync:readmostly:rcu)=
## Read-Copy Update
